In [1]:
from pyspark.sql import SparkSession

In [2]:
MAX_MEMORY = '8g'

spark = SparkSession.builder \
    .appName('recommender_system1') \
    .config('spark.driver.memory', MAX_MEMORY) \
    .config('spark.some.config.option', 'some-value') \
    .getOrCreate()

base_path = '/Users/hyunseokjung/data/movie_dataset/'

print('\n\nLoad Movie Dataset : ratings, movies, links\n\n')
ratings = spark.read.csv(base_path+'ratings.csv', header=True, inferSchema=True).repartition(5).cache()
metadata = spark.read.csv(base_path+'movies_metadata.csv', header=True, inferSchema=True).repartition(5).cache()
links = spark.read.csv(base_path+'links.csv', header=True, inferSchema=True).repartition(5).cache()

ratings = ratings.select('userId', 'movieId', 'rating').cache()
print('Transform : ratings\n\n')
print(ratings.show(3))
print(f'UserId Count : {ratings.count()}')

metadata = metadata.select('imdb_id', 'title', 'vote_average', 'release_date').cache()
print('Transform : movies\n\n')
print(metadata.show(3))
print(f'Movie Count : {metadata.count()}')

Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


22/12/11 16:15:54 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
22/12/11 16:15:54 WARN Utils: Service 'SparkUI' could not bind on port 4040. Attempting port 4041.
22/12/11 16:15:54 WARN Utils: Service 'SparkUI' could not bind on port 4041. Attempting port 4042.


Load Movie Dataset : ratings, movies, links




Transform : ratings




+------+-------+------+
|userId|movieId|rating|
+------+-------+------+
|  1906|   5349|   3.5|
| 16458|    778|   4.0|
| 12642|  60684|   3.5|
+------+-------+------+
only showing top 3 rows

None


UserId Count : 26024289
Transform : movies


+---------+--------------------+------------+------------+
|  imdb_id|               title|vote_average|release_date|
+---------+--------------------+------------+------------+
|tt0093818|          Radio Days|         7.0|  1987-01-30|
|tt0144969|           Home Page|         0.0|  1998-10-14|
|tt0168987|Better Than Choco...|         6.4|  1999-02-14|
+---------+--------------------+------------+------------+
only showing top 3 rows

None
Movie Count : 45572


In [3]:
links.show(3)

+-------+------+------+
|movieId|imdbId|tmdbId|
+-------+------+------+
|  92956|113666| 70374|
| 174759| 17372|189505|
|    418|106379| 46924|
+-------+------+------+
only showing top 3 rows



In [22]:
import pandas as pd

metadata_pd = metadata.toPandas()
links_pd = links.toPandas()

In [25]:
metadata_pd

,imdb_id,title,vote_average,release_date
0,tt0093818,Radio Days,7.0,1987-01-30
1,tt0144969,Home Page,0.0,1998-10-14
2,tt0168987,Better Than Chocolate,6.4,1999-02-14
3,tt0084503,Pink Floyd: The Wall,7.7,1982-07-14
4,tt0238015,All Access: Front Row. Backstage. Live!,0.0,2001-05-20
...,...,...,...,...
45567,tt0060401,None,False,"[{'iso_3166_1': 'PL', 'name': 'Poland'}]"
45568,tt0997282,Solstorm,5.4,2007-11-02
45569,tt4659060,La révolution n'est pas un dîner de gala,0.0,2015-12-01
45570,tt4208868,None,False,"[{'iso_3166_1': 'NL', 'name': 'Netherlands'}]"


In [24]:
links_pd

,movieId,imdbId,tmdbId
0,92956,113666,70374.0
1,174759,17372,189505.0
2,418,106379,46924.0
3,154222,402590,64190.0
4,27869,386064,11658.0
...,...,...,...
45838,153362,4934296,375732.0
45839,139705,2120025,326262.0
45840,3151,20668,26162.0
45841,71216,19946,117531.0


In [26]:
def get_movie_metadata(movieId):
    metadata_pd['imdb_id'] = metadata_pd['imdb_id'].astype('category')
    imdb_id = links_pd[links_pd['movieId'] == movieId]
    imdb_id = imdb_id.imdbId.values[0]
    if len(str(imdb_id)) == 7:
        movie_rated = metadata_pd[metadata_pd['imdb_id'] == 'tt'+imdb_id.astype(str)]
        df = movie_rated.loc[:,['title', 'vote_average', 'release_date']]
        return df.reset_index(drop=True)
    elif len(str(imdb_id)) == 6:
        movie_rated = metadata_pd[metadata_pd['imdb_id'] == 'tt0'+imdb_id.astype(str)]
        df = movie_rated.loc[:,['title', 'vote_average', 'release_date']]
        return df.reset_index(drop=True)
    elif len(str(imdb_id)) == 5:
        movie_rated = metadata_pd[metadata_pd['imdb_id'] == 'tt00'+imdb_id.astype(str)]
        df = movie_rated.loc[:,['title', 'vote_average', 'release_date']]
        return df.reset_index(drop=True)
    elif len(str(imdb_id)) == 4:
        movie_rated = metadata_pd[metadata_pd['imdb_id'] == 'tt000'+imdb_id.astype(str)]
        df = movie_rated.loc[:,['title', 'vote_average', 'release_date']]
        return df.reset_index(drop=True)
    elif len(str(imdb_id)) == 3:
        movie_rated = metadata_pd[metadata_pd['imdb_id'] == 'tt0000'+imdb_id.astype(str)]
        df = movie_rated.loc[:,['title', 'vote_average', 'release_date']]
        return df.reset_index(drop=True)
    elif len(str(imdb_id)) == 2:
        movie_rated = metadata_pd[metadata_pd['imdb_id'] == 'tt00000'+imdb_id.astype(str)]
        df = movie_rated.loc[:,['title', 'vote_average', 'release_date']]
        return df.reset_index(drop=True)
    elif len(str(imdb_id)) == 1:
        movie_rated = metadata_pd[metadata_pd['imdb_id'] == 'tt000000'+imdb_id.astype(str)]
        df = movie_rated.loc[:,['title', 'vote_average', 'release_date']]
        return df.reset_index(drop=True)
    else:
        pass

In [40]:
movie_list = [30, 50, 203, 4973]

df_aux_b = pd.DataFrame({'title': ['aaa'], 
                         'vote_average': [1.7], 
                         'release_date': ['1999-01-01']
        })

for movie_id in movie_list:
    df_aux_b = df_aux_b.append(get_movie_metadata(movie_id), ignore_index=True)

/var/folders/b9/8j1qkty17r9244bwmgfs9kdc0000gn/T/ipykernel_24489/2514556784.py:9: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df_aux_b = df_aux_b.append(get_movie_metadata(movie_id), ignore_index=True)
/var/folders/b9/8j1qkty17r9244bwmgfs9kdc0000gn/T/ipykernel_24489/2514556784.py:9: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df_aux_b = df_aux_b.append(get_movie_metadata(movie_id), ignore_index=True)
/var/folders/b9/8j1qkty17r9244bwmgfs9kdc0000gn/T/ipykernel_24489/2514556784.py:9: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df_aux_b = df_aux_b.append(get_movie_metadata(movie_id), ignore_index=True)
/var/folders/b9/8j1qkty17r9244bwmgfs9kdc0000gn/T/ipykernel_24489/2514556784.py:9: FutureWarning: The frame.append method 

In [47]:
for movie_id in movie_list:
    df_aux_b = pd.concat([df_aux_b, get_movie_metadata(movie_id)])

In [50]:
pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)
pd.set_option('display.width', None)
pd.set_option('display.max_colwidth', -1)

print(df_aux_b)

                                              title vote_average release_date
0  aaa                                               1.7          1999-01-01 
1  Shanghai Triad                                    6.5          1995-04-30 
2  The Usual Suspects                                8.1          1995-07-19 
3  To Wong Foo, Thanks for Everything! Julie Newmar  6.3          1995-09-07 
4  Amélie                                            7.8          2001-04-25 
0  Shanghai Triad                                    6.5          1995-04-30 
0  The Usual Suspects                                8.1          1995-07-19 
0  To Wong Foo, Thanks for Everything! Julie Newmar  6.3          1995-09-07 
0  Amélie                                            7.8          2001-04-25 


/var/folders/b9/8j1qkty17r9244bwmgfs9kdc0000gn/T/ipykernel_24489/2805075369.py:4: FutureWarning: Passing a negative integer is deprecated in version 1.0 and will not be supported in future version. Instead, use None to not limit the column width.
  pd.set_option('display.max_colwidth', -1)


22/12/11 19:55:53 WARN HeartbeatReceiver: Removing executor driver with no recent heartbeats: 275986 ms exceeds timeout 120000 ms
22/12/11 19:55:53 WARN SparkContext: Killing executors is not supported by current scheduler.


In [34]:
get_movie_metadata(4973)

,title,vote_average,release_date
0,Amélie,7.8,2001-04-25


In [ ]:
df_aux_b = pd.DataFrame({'title': ['aaa'], 
                         'vote_average': [1.7], 
                         'release_date': ['1999-01-01']
        })

for i in movieIdIdx:
    df_aux_b = df_aux_b.append(get_movie_metadata(i), ignore_index=True)

In [45]:
df_aux_b

,title,vote_average,release_date
0,aaa,1.7,1999-01-01
1,Shanghai Triad,6.5,1995-04-30
2,The Usual Suspects,8.1,1995-07-19
3,"To Wong Foo, Thanks for Everything! Julie Newmar",6.3,1995-09-07
4,Amélie,7.8,2001-04-25


In [46]:
for i in movieIdIdx:
    df_aux_b = pd.concat([df_aux_b, get_movie_metadata(i)])

NameError: name 'movieIdIdx' is not defined

In [21]:
# metadata_link = metadata.join(links, "metadata.*imdb_id = links.imdbId")
# metadata_link.show(3)

# select('imdb_id').show()

AnalysisException: USING column `metadata.*imdb_id = links.imdbId` cannot be resolved on the left side of the join. The left-side columns: [imdb_id, title, vote_average, release_date]